### Initial Environment

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from myexptorch import *

### Load scData initialization

In [2]:
# df = pd.read_csv('processfile/scale_df.csv',index_col=0)
# h5 = pd.HDFStore('processfile/scale_df.h5','w', complevel=4, complib='blosc')
# h5['data'] = df
# h5.close()
df = pd.read_hdf('../processfile/scale_df.h5',key='data')
df = df.T

# make map to lock every sample
cellinfo = pd.DataFrame(df.index,index=df.index,columns=['cell_index'])
cellinfo['id'] = range(1,df.shape[0]+1)

# create x, y
x = expToTorch(df)
y = labelTotorch(cellinfo.id)    
train_iter = makeDataiter(x,y,batch_szie=1024)

In [15]:
class AutoEncoder(pl.LightningModule):
    # 1. 初始化模型
    def __init__(self):
        super(AutoEncoder,self).__init__()
        self.encoder  =  nn.Sequential(
            nn.Linear(2000,1000),
            nn.ReLU(),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Linear(500, 250),
            nn.ReLU(),
            nn.Linear(250, 125)
            )
        
        self.decoder = nn.Sequential(
            nn.Linear(125,250),
            nn.ReLU(),
            nn.Linear(250, 500),
            nn.ReLU(),
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Linear(1000,2000)
            )
              
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded,decoded
    
    # 2. 定义损失函数
    def myloss(self, decoded, labels):
        return F.mse_loss(decoded, labels) # 不可以使用nn模块
    
    # 3. 定义优化器
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    # 4. 训练loop
    def training_step(self, train_batch, batch_ix):
        x, y = train_batch
        encoded,decoded = self.forward(x)
        loss = self.myloss(decoded, x)
        
        logs = {'train_loss': loss} # 记录每一次训练的loss
        return {'loss' : loss, 'log':logs}
    
    ## 5. 验证loop
    #def validation_step(self, val_batch, batch_ix):
    #    x, y = val_batch
    #    logist = self.forward(x)
    #    loss = self.myloss(logis, x)
    #    return {'val' : loss}
    #
    #def validation_step_end(self, output):
    #    # 验证后调用
    #    # 这里的output是由上一步的输出，是一个array类型
    #    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #    tensorboard_logs = {'val_loss': avg_loss} 
    #    return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

In [16]:
model = AutoEncoder()
trainer = pl.Trainer(gpus=-1,auto_scale_batch_size= True)
trainer.fit(model, train_iter)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 2.7 M 
1 | decoder | Sequential | 2.7 M 
---------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params


1